# **🎯 GPT 기반 정량평가 : : QG-QA**
## **📍문제 정의**
- 뉴스 기사 각각에 대해 LLM을 적용하여 기사에 대한 블로그 글을 만든 상황에서,
- 블로그 글이 뉴스 기사의 내용을 잘 담고 있는지 QG-QA를 통해 평가하고자 한다.
- 사용 모델: **gpt-4o**

## **📍수행 과정**
1. **뉴스 기사와 그에 대응하는 블로그 글에 대해 동일한 질문을 생성**
  - OUTPUT 이 질문 리스트가 되도록 PROMPT 구성하기
2. **뉴스 기사, 블로그 글에 질문 각각 ASK**
3. **그에 대한 답변이 서로 일치하는지 확인한다.**
  - 두 답변을 서로 비교하여 일치할 경우 True(1), 아닐 경우 False(0) 반환

## **📍선택지**
1. **한번에 한개의 질문만 / 한번에 여러개의 질문?** → *여러개의 질문*
2. **페르소나 1개로 통일/질문답변 페르소나 2개로 분리?** → *2개*
3. **메모리 사용/미사용?** → *미사용*

## **📍기타사항**
1. 질문 2개로 축소(사건 발생 원인은 다른 질문에 비해 중요성과 정확성이 낮아 제거함)
2. 질문마다 5개의 선지, '알 수 없음; 선지는 무조건 포함
3. QG와 QA단계 모두에 Step 을 지정함
4. gpt-4-turbo 대신 gpt-4o 사용 - 비용 절감
5. ***🚨해당 함수에 뉴스와 블로그 데이터를 주입할 때 '반드시' 뉴스는 제목, 발행일자, 본문이 분리된 상태일 것***

## **📍진행 순서**

        * (1) 입력 - 뉴스 제목, 뉴스 본문, 뉴스 날짜, 블로그 텍스트 → news_sample, blog_sample
        * (2) RPOMPT1 - 뉴스기사를 기반으로 질문을 만들어줘- 질문 도출
        * (3) PROMPT2 - 뉴스 기사에 대해 질문의 답변 도출
        * (4) PROMPT3 - 블로그 글에 대해 질문의 답변 도출
        * (5)일치여부판단 - (3), (4)의 결과로 나온 답변을 비교

# **📍1. 환경세팅**

In [1]:
!pip install --upgrade openai
!pip install langchain
!pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.4 MB/s eta 0:00:00


In [2]:
import getpass
import os
# 환경변수에 OpenAI API 키 저장 (사용자 입력으로 안전하게)
os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [6]:
# GPT-4 챗봇 객체 생성
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

# 프롬프트 템플릿, 파서를 각각 생성한 뒤 챗봇과 체인 결합
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

# PROMPT1 - 뉴스 기반으로 질문 생성 - chain_1으로 생성
prompt_1 = ChatPromptTemplate.from_messages([
    ("system", """You are an AI trained to generate insightful questions from a given article."""), # 페르소나 부여
    ("user", """
    1. Read the article about current Seoul subway news.
    2. Find out what is the main incident related with subway in the article.
    3. Find the date that this article was issued
    4. Find every date-related expression in the article.
    5. Compare 3 and 4, and find out the date the incident was occured.
    6. Find every expression related to line of subway in the article.
    7. Find out which line the incident is about.
    8. Create 2 questions to identify the main points of the news (date of the incident, subway line of the incident). The questions should be 5-way multiple choice questions where you have to choose one of the choices from 1 to 5. One of the options must be “Unknown” and the selection for the “Date of Event” question must be in the format 2018년 3월 18일. Hint is provided with every questions

    <Example Question>: 'Where did the subway incident occur? (1) Gangnam Station (2) Seongsu Station (3) Suyu Station (4) Ankguk Station (5) Unknown)',
    <Example Hint>:''
    News Article= {input}
    """),
    ("system", """Generate 2 questions. All should be in Korean ONLY.

Template:
Question1: {{}},
Hint1: {{}},
Question2: {{}},
Hint2: {{}}""")
])
chain_1 = prompt_1 | llm | output_parser

# PROMPT2 입력 - 뉴스 기사에 대해 질문의 답변 도출 - chain_2으로 생성
prompt_2 = ChatPromptTemplate.from_messages([
    ("system", """You read a news article and answer a question accurately based on what you read."""), # 페르소나 부여
    ("user", """
    You read a news article like this:
    1. Read the article about current Seoul subway news.
    2. Find out what is the main incident related with subway in the article.
    3. Find the date that this article was issued
    4. Find every date-related expression in the article.
    5. Compare 3 and 4, and find out the date the incident was occured.
    6. Find every expression related to line of subway in the article.
    7. Find out which line the incident is about.
    Then you answer a question accurately based on what you read.
    Example= “1번, 5번, 4번”,
    news= {input}
    Questions= {question}
    """),
    ("system", """"Template(MUST FOLLOW): Answer1: {{answer_1}}번, Answer2: {{answer_2}}번""")
])
chain_2 = prompt_2 | llm | output_parser

# PROMPT3 입력 - 블로그글에 대해 질문의 답변 도출 - chain_3으로 생성
prompt_3 = ChatPromptTemplate.from_messages([
    ("system", """You read a blog article and answer a question accurately based on what you read"""), # 페르소나 부여
    ("user", """
    You read a blog article like this:
    1. Read the blog about current Seoul subway news.
    2. Find out what is the main incident related with subway in the article.
    3. Find every date-related expression in the article.
    4. Find out the date the incident was occured.
    5. Find every expression related to line of subway in the article.
    6. Find out which line the incident is about.
    Then you answer 2 questions accurately based on what you read.
    Example= “1번, 5번, 4번”,
    blog= {input}
    Questions= {question}
    """),
    ("system", """Template(MUST FOLLOW): Answer1: {{answer_1}}번, Answer2: {{answer_2}}번""")
])
chain_3 = prompt_3 | llm | output_parser



def quan_eval(title, date, article, blog):

  # news = 제목 + 날짜 + 본문
  news = f"""<뉴스 제목>: {news_title},
  <뉴스 생성일자>: {news_date},
  <뉴스 본문>: {news_article}"""

  # news 기반으로 질문 3개 생성
  question_list = chain_1.invoke({"input": news})

  # 질문 3개에 대해 news 기반으로 답변
  answer_list_news = chain_2.invoke({"input": news, "question": question_list})

  # 질문 3개에 대해 blog 기반으로 답변
  answer_list_blog = chain_3.invoke({"input": blog, "question": question_list})

  # 일치 여부 판단(True/False)
  return answer_list_news == answer_list_blog

# **적용 예시**
## **데이터 준비**

In [4]:
# [뉴스]

# 제목
news_title = '"전쟁난 줄 알았다"…출근길 직장인 울린 서울 지하철 상황'
# 날짜
news_date = '2023-07-14 00:00:00'
# 본문
news_article = """\수인분당선 왕십리역에서 열차를 기다리는 시민들이 대거 몰려있는 모습. /사진=김세린 기자 14일 서울에 많은 비가 내리면서 일부 도로의 출입이 전면 통제된 가운데, 출근길 지하철로 몰린 시민들이 곳곳에서 불편함을 호소했다. 이날 오전 4시10분께 출입이 통제된 동부간선도로 전 구간(수락지하차도~성수JC)은 오전 6시 40분께 통행이 재개됐으나, 오전 7시 15분께부터는 올림픽대로(양방향) 여의상류IC 교통 통제가 시작됐다. 서울시 재난안전대책본부는 시민들에게 미리 도로 교통 상황을 확인하고, 가급적 대중교통을 이용해 달라고 당부했다. 이에 일부 지하철 노선에 사람이 대거 몰리게 된 것으로 풀이된다. 수서역에서 왕십리역 방향 열차 지연으로 대기 중이던 시민들이 열차 탑승을 기다리고 있다. /사진=김세린 기자 이날 오전 8시께 수인 분당선 수서역에서 왕십리로 향하는 열차는 운행이 10~15분가량 지연돼 다음 역인 대모산입구역에서 기다리던 승객들이 대기해야 했다. 이후 재개된 열차는 5대가량이 연이어 몰려오는 상황이 발생했으며, 도착한 열차 내부엔 사람들이 이미 꽉 들어서 한때 원활한 탑승이 어려웠다. 이 구간 종착역인 왕십리에서도 청량리 방면 열차 탑승을 위해 대기 중인 사람들이 빼곡하게 줄지어 들어서 있었다. 시민들 사이에서는 "전쟁 난 거 아니냐", "움직일 수가 없다", "오늘 안에 가는 것 맞냐"는 등의 목소리가 흘러나왔다. 직장인 송모 씨(42)는 "비가 많이 와서 도로 상황이 안 좋은 것 같아 지하철을 타러 온 건데 이게 무슨 일인지 당황스럽다"며 "회사 가는 건 이미 지각이고, 지옥철이라서 불편할 것 같다"고 토로했다. 지하철 운행이 폭우로 인해 늦어진 것과 관련, 직장인 온라인 커뮤니티 '블라인드'에서도 "오늘 2호선 역마다 몇분씩 정차하는 것이냐", "출근 시간에 지하철 왜 이러는 거냐", "오늘 지하철 왜 이러냐 지각 당첨이다" 등의 반응이 나왔다. 집중호우가 내린 14일 오전 서울 잠수교가 강물에 잠겨 통제되고 있다. /사진=뉴스1 이외에도 폭우로 인한 피해도 계속된다. 밤사이 세찬 비가 이어지면서 서울 2개 구 4000여세대에서 정전 피해가 발생했으며, 전국 6개 시도 21개 시군구 134명이 일시 대피했다. 한편 기상청에 따르면 오늘까지 서울과 인천, 경기 북부, 강원중·북부 내륙·산지에 돌풍과 천둥·번개를 동반한 시간당 30∼80mm의 매우 강한 비가 내린다. 행정안전부는 위기 경보 수준을 '경계'에서 최고 수준인 '심각' 단계로 상향하고 중앙재난안전대책본부(중대본) 2단계를 3단계로 올리기로 했다.""" # 본문




# [블로그]

blog_sample = """
## "폭우에 막힌 서울의 아침: 지하철 연착으로 출근길 뒤숭숭"

## 시작하는 말:
안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!

-{지연/사고 일시}: 2023년 7월 14일 오전
-{지연/사고 노선}: 수인분당선
-{지연/사고 이유}: 강한 폭우로 인한 노선 일부 구간의 도로 출입 통제 및 대체 대중교통 수요 급증
-{문의 사항}: [서울교통공사 홈페이지](https://www.seoulmetro.co.kr)

## 간단한 요약글:
지난 14일, 서울을 포함한 수도권에 내린 집중 호우로 인해 많은 도로가 통제되었고, 대중교통 이용에 큰 차질이 빚어졌습니다. 특히 수인분당선에서는 수서역에서 왕십리역 방향으로 향하는 열차가 평소보다 10~15분 가량 지연되었으며, 이로 인해 승객들이 열차 내부에서 긴 대기 시간을 겪어야 했습니다. 또한, 청량리 방향으로 가는 열차를 기다리는 승객들로 인해 왕십리역이 빼곡히 들어차 원활한 탑승이 어려운 상황이었습니다. 이러한 상황은 출근 시간대에 직장인들 사이에서 큰 불편을 초래했습니다.

## 마무리 말:
오지통이 실시간으로 다양한 지하철 정보를 업데이트 할 예정이니, 자주 방문해 주세요. '지하철 온다'는 단 한 번의 터치로 자신의 위치에서 가장 가까운 지하철역의 실시간 도착 정보를 제공합니다. 아침마다 이런 사태로 지각이 걱정되시죠? 우리 함께 잘 대비해봅시다!

🔽 지하철 온다 소개 보러가기

https://blog.naver.com/subway__onda/223258646349"""

In [8]:
quan_eval(news_title, news_date, news_article, blog_sample)

True